## Parser / Python's html.parser

In [16]:
from bs4 import BeautifulSoup
html = ''' 
<ol>
    <li>NEVER - 국민의 아들</li>
    <li>SIGNAL - TWICE</li>
    <li>LONELY - 씨스타</li>
    <li>I LUV IT - PSY</li>
    <li>New Face - PSY</li>
</ol>
'''
soup = BeautifulSoup(html, 'html.parser')
for tag in soup.select('li'):
    print(tag.text)

NEVER - 국민의 아들
SIGNAL - TWICE
LONELY - 씨스타
I LUV IT - PSY
New Face - PSY


## Parser / lxml's HTML parser

In [17]:
from bs4 import BeautifulSoup
html = ''' 
<ol>
    <li>NEVER - 국민의 아들</li>
    <li>SIGNAL - TWICE</li>
    <li>LONELY - 씨스타</li>
    <li>I LUV IT - PSY</li>
    <li>New Face - PSY</li>
</ol>
'''
soup = BeautifulSoup(html, 'lxml')
for tag in soup.select('li'):
    print(tag.text)

NEVER - 국민의 아들
SIGNAL - TWICE
LONELY - 씨스타
I LUV IT - PSY
New Face - PSY


---

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

# 로그 기록 함수
def log_message(message):
    timestamp = datetime.datetime.now().strftime('%Y-%b-%d-%H-%M-%S')
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp}, {message}\n")
        

# Extract
def extract_gdp_data(url):
    log_message("Starting Data Extraction")
    # HTTP 요청 후 파싱해서 데이터 추출
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    rows = table.find_all('tr')

    data = []
    # 불필요한 (헤더) 제거 후 데이터 추출
    for row in rows[3:]:
        cols = row.find_all(['td', 'th'])
        
        # 'sup' 태그 제거 후 텍스트 추출
        cleaned_cols = []
        for col in cols:
            for sup in col.find_all('sup'):
                sup.decompose()  # 'sup' 태그 제거
            cleaned_cols.append(col.text.strip())
        
        data.append(cleaned_cols)
    return data
    

    
# Transfrom
def transfrom_gdp_data(data):
    log_message("Starting Data Transformation")
    # 국가별 GDP 확인 가능한 테이블 만들기
    df = pd.DataFrame(data)
    # 열 수 확인해서 조건 맞아야만 실행
    if len(df.columns) == 7:
        df.columns = df.columns = ['Country', 'IMF_Forecase', 'IMF_Year', 'WB_Forecase', 'WB_Year', 'UN_Forecase', 'UN_Year']
        # 필요한 값만 추출 (일단은 IMF만)
        df = df[['Country', 'IMF_Forecase', 'IMF_Year']]
    else:
        raise ValueError(f"Unexpected number of columns: {len(df.columns)}")
    
    
    
    
    
    
# Load

# 메인 ETL 함수
def etl_process():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
    try:
        log_message("ETL Process Started")
        data = extract_gdp_data(url)
        log_message("ETL Process End Successfully")
    except Exception as e:
        log_message(f"ETL Process Failed: {str(e)}")
        


if __name__ == "__main__":
    etl_process()

SyntaxError: invalid syntax (1819971974.py, line 40)

만들어야 하는 것 
1. 국가별 GDP 확인 가능한 테이블 (GDP가 높은 국가들 순서로/ 단위는 1B USD로 소수점 2자리/ 갱신해도 재사용 가능하게)
2. <u>로그 기록시 datetime Year-Monthname-Day-Hour-Minute-Second (각 단계의 시작과 끝을 로그에 기록(기족 파일에 append))</u>
3. 추출 (Extract)한 정보는 'Countries_by_GDP.json'라는 이름의 JSON 화일 포맷으로 저장
4.  필요한 모든 작업을 수행하는 'etl_project_gdp.py' 코드를 작성

지켜야 하는 것
1. 주석 사용
2. 함수 만들어 재사용성

화면 출력
1. GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력
2. 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력


---

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"

# HTTP 요청 후 파싱해서 데이터 추출
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'class': 'wikitable'})
rows = table.find_all('tr')

data = []
# 불필요한 (헤더) 제거 후 데이터 추출
for row in rows[3:]:
    cols = row.find_all(['td', 'th'])
    
    # 'sup' 태그 제거 후 텍스트 추출
    cleaned_cols = []
    for col in cols:
        for sup in col.find_all('sup'):
            sup.decompose()  # 'sup' 태그 제거
        cleaned_cols.append(col.text.strip())
    
    data.append(cleaned_cols)


# 결과 출력
for row in data[:8]:  # 일부 데이터만 출력
    print(row)

['United States', '30,337,162', '2025', '27,360,935', '2023', '25,744,100', '2022']
['China', '19,534,894', '2025', '17,794,782', '2023', '17,963,170', '2022']
['Germany', '4,921,563', '2025', '4,456,081', '2023', '4,076,923', '2022']
['Japan', '4,389,326', '2025', '4,212,945', '2023', '4,232,173', '2022']
['India', '4,271,922', '2025', '3,549,919', '2023', '3,465,541', '2022']
['United Kingdom', '3,730,261', '2025', '3,340,032', '2023', '3,089,072', '2022']
['France', '3,283,429', '2025', '3,030,904', '2023', '2,775,316', '2022']
['Italy', '2,459,597', '2025', '2,254,851', '2023', '2,046,952', '2022']


In [35]:
data = [    
    ['United States', '30,337,162', '2025', '27,360,935', '2023', '25,744,100', '2022'],
    ['China', '19,534,894', '2025', '17,794,782', '2023', '17,963,170', '2022'],
    ['Germany', '4,921,563', '2025', '4,456,081', '2023', '4,076,923', '2022'],
    ['Japan', '4,389,326', '2025', '4,212,945', '2023', '4,232,173', '2022'],
    ['India', '4,271,922', '2025', '3,549,919', '2023', '3,465,541', '2022'],
    ['United Kingdom', '3,730,261', '2025', '3,340,032', '2023', '3,089,072', '2022']
]   

df = pd.DataFrame(data)
df.columns = ['Country', 'IMF_Forecase', 'IMF_Year', 'WB_Forecase', 'WB_Year', 'UN_Forecase', 'UN_Year']
print(df)
print('-' * 70)
df = df[['Country', 'IMF_Forecase', 'IMF_Year']]
print(df)

          Country IMF_Forecase IMF_Year WB_Forecase WB_Year UN_Forecase  \
0   United States   30,337,162     2025  27,360,935    2023  25,744,100   
1           China   19,534,894     2025  17,794,782    2023  17,963,170   
2         Germany    4,921,563     2025   4,456,081    2023   4,076,923   
3           Japan    4,389,326     2025   4,212,945    2023   4,232,173   
4           India    4,271,922     2025   3,549,919    2023   3,465,541   
5  United Kingdom    3,730,261     2025   3,340,032    2023   3,089,072   

  UN_Year  
0    2022  
1    2022  
2    2022  
3    2022  
4    2022  
5    2022  
----------------------------------------------------------------------
          Country IMF_Forecase IMF_Year
0   United States   30,337,162     2025
1           China   19,534,894     2025
2         Germany    4,921,563     2025
3           Japan    4,389,326     2025
4           India    4,271,922     2025
5  United Kingdom    3,730,261     2025
